In [1]:
import pandas as pd
from collections import Counter

## First step: Exploring the data
Reading in the file and checking the different data types to get inspiration about visualization options.

In [4]:
dataset = pd.read_csv('/content/drive/MyDrive/OPINION_COST/2._gllm-annotation-results.csv')

In [11]:
valid_ids = dataset[(dataset['variable'] == 'Q1_0_Tool-Mentioned') & (dataset['result'] == 'Yes')]['id']

In [14]:
dataset = dataset[dataset['id'].isin(valid_ids)]

In [69]:
def count_values(dataset, variable_name):
    """
    Count the occurrences of unique values in a specific variable from the dataset.

    Parameters:
    - dataset (pd.DataFrame): The dataset containing the data.
    - variable_name (str): The name of the variable to process.

    Returns:
    - list: A sorted list of tuples with values and their counts.
    """
    # Filter the dataset for the specified variable
    values = dataset[dataset['variable'] == variable_name]['result'].tolist()

    # Process the values into a list of individual items
    value_list = [v.lower().split('; ') for v in values]

    # Count the occurrences of each unique value
    value_counter = Counter(x for xs in value_list for x in set(xs))

    # Return the most common values
    return value_counter.most_common()

# Example usage:
# result = count_values(dataset, 'Q5_3_Data-Country')
# print(result)


In [71]:
opinion_evaluation = count_values(dataset, 'Q4_1_Opinion-Evaluation')

In [72]:
toolname = count_values(dataset, 'Q1_1_Tool-Name')

In [73]:
approach = count_values(dataset, 'Q3_1_Approach')

In [74]:
target = count_values(dataset, 'Q4_2_Opinion-Target')

In [75]:
source = count_values(dataset, 'Q5_1_Data-Source')

In [76]:
language = count_values(dataset, 'Q5_2_Data-Language')

In [77]:
countries = count_values(dataset, 'Q5_3_Data-Country')

In [86]:
toolname2 = count_values(dataset, 'Q2_1_Tool-Name')

In [87]:
dataset_name = count_values(dataset, 'Q5_4_Dataset-Name')